### Helper to push and pull from pi


In [3]:
import paramiko
import os

# Raspberry Pi SSH Credentials: 
RASPBERRY_PI_IP = "100.73.143.76"  # Replace with your actual Pi's IP
USERNAME = "shah"  # Your Pi username
PASSWORD = "123456"  # If using SSH key, remove this
REMOTE_DIR = "/home/shah/robot_control"  # Pi directory
LOCAL_DIR = os.getcwd()  # Automatically use the current directory

# Initialize SSH & SFTP clients
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    print("\n🔗 Connecting to Raspberry Pi...")
    ssh.connect(RASPBERRY_PI_IP, username=USERNAME, password=PASSWORD)

    sftp = ssh.open_sftp()
    
    def pull_files():
        """Download all files from the Pi's robot_control folder to Windows (current directory)."""
        print("\n📥 Pulling files from Raspberry Pi to your local directory:")
        print(f"   Remote: {REMOTE_DIR}")
        print(f"   Local : {LOCAL_DIR}\n")

        for file in sftp.listdir(REMOTE_DIR):
            remote_path = f"{REMOTE_DIR}/{file}"
            local_path = os.path.join(LOCAL_DIR, file)

            # Check if it's a file (skip directories like __pycache__)
            try:
                if sftp.stat(remote_path).st_mode & 0o170000 == 0o100000:  # Regular file
                    print(f"  ✅ Downloading {file}...")
                    sftp.get(remote_path, local_path)
                else:
                    print(f"  🚫 Skipping directory: {file}")
            except Exception as e:
                print(f"  ❌ Error downloading {file}: {e}")
        
        print("\n✅ Pull complete! All files are now in your local directory.")

    def push_files():
        """Upload specific files from the current directory to Raspberry Pi."""
        print("\n📂 Current directory:")
        print(f"   {LOCAL_DIR}\n")
        
        files = os.listdir(LOCAL_DIR)
        files = [f for f in files if os.path.isfile(f)]  # Only show files, not directories
        
        if not files:
            print("❌ No files found in the current directory.")
            return

        # Show available files
        print("📜 Available files to upload:")
        for idx, file in enumerate(files):
            print(f"  [{idx + 1}] {file}")

        # Let the user choose files to upload
        print("\n🔹 Enter the file numbers you want to push (comma-separated), or type 'all' to upload everything.")
        print("   Example: '1,3' will upload files 1 and 3.")
        print("   Example: 'all' will upload all files.")
        
        selections = input("\n📤 Enter your choice: ").strip().lower()

        if selections == "all":
            selected_files = files
        else:
            try:
                selected_files = [files[int(i) - 1] for i in selections.split(",") if i.isdigit()]
            except (IndexError, ValueError):
                print("❌ Invalid selection. Please enter valid file numbers.")
                return

        print(f"\n⬆️ Uploading selected files to Raspberry Pi ({REMOTE_DIR}):")
        for file in selected_files:
            local_path = os.path.join(LOCAL_DIR, file)
            remote_path = f"{REMOTE_DIR}/{file}"

            try:
                print(f"  ✅ Uploading {file}...")
                sftp.put(local_path, remote_path)
            except Exception as e:
                print(f"  ❌ Error uploading {file}: {e}")

        print("\n✅ Push complete! Selected files have been uploaded.")

    # Choose whether to pull or push
    print("\n🔹 Available actions:")
    print("   - Type 'pull' to **download** files from Raspberry Pi to your computer.")
    print("   - Type 'push' to **upload** files from your computer to Raspberry Pi.")
    
    action = input("\n🔄 Enter 'pull' or 'push': ").strip().lower()
    
    if action == "pull":
        pull_files()
    elif action == "push":
        push_files()
    else:
        print("❌ Invalid option. Please enter 'pull' or 'push'.")

except Exception as e:
    print(f"❌ Error: {e}")

finally:
    sftp.close()
    ssh.close()
    print("\n🔌 Disconnected from Raspberry Pi.")



🔗 Connecting to Raspberry Pi...

🔹 Available actions:
   - Type 'pull' to **download** files from Raspberry Pi to your computer.
   - Type 'push' to **upload** files from your computer to Raspberry Pi.



🔄 Enter 'pull' or 'push':  push



📂 Current directory:
   C:\Users\Hat_Shed\Desktop\490-Testing Robot\robot_control

📜 Available files to upload:
  [1] helper.ipynb
  [2] Motor.py
  [3] robot_server.py
  [4] servo.py
  [5] __pycache__

🔹 Enter the file numbers you want to push (comma-separated), or type 'all' to upload everything.
   Example: '1,3' will upload files 1 and 3.
   Example: 'all' will upload all files.



📤 Enter your choice:  2



⬆️ Uploading selected files to Raspberry Pi (/home/shah/robot_control):
  ✅ Uploading Motor.py...

✅ Push complete! Selected files have been uploaded.

🔌 Disconnected from Raspberry Pi.


### Helper To run remote server:

In [9]:
import paramiko

# Raspberry Pi SSH Credentials
RASPBERRY_PI_IP = "100.73.143.76"  # Update with your actual Pi IP
USERNAME = "shah"  # Your Pi username
PASSWORD = "123456"  # If using SSH key, remove this
REMOTE_DIR = "/home/shah/robot_control"  # Pi directory

# Initialize SSH client
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    print("\n🔗 Connecting to Raspberry Pi...")
    ssh.connect(RASPBERRY_PI_IP, username=USERNAME, password=PASSWORD)

    # Command to start robot_server.py in the background
    start_command = f"cd {REMOTE_DIR} && nohup python3 robot_server.py > server_log.txt 2>&1 &"
    
    print("🚀 Starting robot server on Raspberry Pi...")
    stdin, stdout, stderr = ssh.exec_command(start_command)

    # Check if there were any errors
    error_output = stderr.read().decode().strip()
    if error_output:
        print(f"❌ Error: {error_output}")
    else:
        print("Please ssh and run sudo pigpiod in terminal")

except Exception as e:
    print(f"❌ Error: {e}")

finally:
    ssh.close()
    print("🔌 Disconnected from Raspberry Pi.")



🔗 Connecting to Raspberry Pi...
🚀 Starting robot server on Raspberry Pi...
Please ssh and run sudo pigpiod in terminal
🔌 Disconnected from Raspberry Pi.
